In [36]:
import pandas as pd
import numpy as np
import os

data_site = pd.read_excel(r'C:\Users\Qing Ye\Downloads\Movement Type.xlsb', sheet_name='WITH SITE INFO', engine='pyxlsb')
data_all = pd.read_excel(r'C:\Users\Qing Ye\Downloads\Movement Type.xlsb', sheet_name='WITHOUT SITE SPECIFIC', engine='pyxlsb')

df_site = pd.DataFrame(data_site)
df_all = pd.DataFrame(data_all)

print('df_site', df_site)
print('df_all', df_all)

df_site     Source Movement Type                   Description  Use  Site
0      EWM          1010                       Putaway  yes   Feg
1      EWM          1010                       Putaway  yes  IDAP
2      EWM          1011  Putaway with Storage Process   no   Feg
3      EWM          1011  Putaway with Storage Process   no  IDAP
4      EWM          1012        Putaway (Distributive)  yes   Feg
..     ...           ...                           ...  ...   ...
735     WM           ZY2          RE all acct.assignB5   no   All
736     WM           ZZ1              GR goods receipt   no   All
737     WM           ZZ2          GR for PO   reversal   no   All
738     WM           ZZ3              GR goods receipt   no   All
739     WM           ZZ4             GR TP dely w. tax   no   All

[740 rows x 5 columns]
df_all     Source Movement Type                       Description Selection
0      EWM          1010                           Putaway       yes
1      EWM          1011      P

In [37]:
""" rename columns """

df_site.rename(columns={'Source': 'System', 'Movement Type': 'MvtType',  'Use': 'MvtInScopeTptCalc'}, inplace=True)
df_all.rename(columns={'Source': 'System', 'Movement Type': 'MvtType', 'Selection': 'MvtInScopeTptCalc'}, inplace=True)

print('df_site', df_site)
print('df_all', df_all)

df_site     System MvtType                   Description MvtInScopeTptCalc  Site
0      EWM    1010                       Putaway               yes   Feg
1      EWM    1010                       Putaway               yes  IDAP
2      EWM    1011  Putaway with Storage Process                no   Feg
3      EWM    1011  Putaway with Storage Process                no  IDAP
4      EWM    1012        Putaway (Distributive)               yes   Feg
..     ...     ...                           ...               ...   ...
735     WM     ZY2          RE all acct.assignB5                no   All
736     WM     ZZ1              GR goods receipt                no   All
737     WM     ZZ2          GR for PO   reversal                no   All
738     WM     ZZ3              GR goods receipt                no   All
739     WM     ZZ4             GR TP dely w. tax                no   All

[740 rows x 5 columns]
df_all     System MvtType                       Description MvtInScopeTptCalc
0      EWM    

In [38]:
""" change yes/no to true/false """


df_site['MvtInScopeTptCalc'] = df_site['MvtInScopeTptCalc'].replace({'yes': 'Y', 'no': 'N'})
df_all['MvtInScopeTptCalc'] = df_all['MvtInScopeTptCalc'].replace({'yes': 'Y', 'no': 'N'})

print('df_site', df_site)
print('df_all', df_all)

df_site     System MvtType                   Description MvtInScopeTptCalc  Site
0      EWM    1010                       Putaway                 Y   Feg
1      EWM    1010                       Putaway                 Y  IDAP
2      EWM    1011  Putaway with Storage Process                 N   Feg
3      EWM    1011  Putaway with Storage Process                 N  IDAP
4      EWM    1012        Putaway (Distributive)                 Y   Feg
..     ...     ...                           ...               ...   ...
735     WM     ZY2          RE all acct.assignB5                 N   All
736     WM     ZZ1              GR goods receipt                 N   All
737     WM     ZZ2          GR for PO   reversal                 N   All
738     WM     ZZ3              GR goods receipt                 N   All
739     WM     ZZ4             GR TP dely w. tax                 N   All

[740 rows x 5 columns]
df_all     System MvtType                       Description MvtInScopeTptCalc
0      EWM    

In [39]:
""" filter site/all, merge two all dfs into one all df """

df_all_one = df_site[df_site['Site'] == "All"]
df_site = df_site[df_site['Site'] != "All"]

df_all_one = df_all_one.drop(columns=['Site'])
df_all = pd.concat([df_all, df_all_one], ignore_index=True)

print('df_site', df_site)
print('df_all', df_all)

df_site     System MvtType                   Description MvtInScopeTptCalc        Site
0      EWM    1010                       Putaway                 Y         Feg
1      EWM    1010                       Putaway                 Y        IDAP
2      EWM    1011  Putaway with Storage Process                 N         Feg
3      EWM    1011  Putaway with Storage Process                 N        IDAP
4      EWM    1012        Putaway (Distributive)                 Y         Feg
..     ...     ...                           ...               ...         ...
235    PMX     557                           NaN                 Y  Fegersheim
236    PMX     557                           NaN                 Y         IPM
237    PMX     557                           NaN                 Y       Sesto
238    PMX     557                           NaN                 Y      Suzhou
239    PMX     559                           NaN                 N  Alcobendas

[240 rows x 5 columns]
df_all      System M

In [40]:
import urllib
from sqlalchemy import create_engine, inspect

DATABASES = {
  'default': {
     'ENGINE': 'mssql',
     'NAME': "miebachcapacitytooldb",
     'USER': 'miebachindianapolis',
     "PASSWORD": 'Miebach1',
     "HOST": 'miebachinternalprojectdbs.database.windows.net',
     "PORT": "1433",
     "OPTIONS": {"driver": "ODBC Driver 17 for SQL Server",
             },
     }
}

def connect_database():

    driver = DATABASES['default']['OPTIONS']['driver']
    server = DATABASES['default']['HOST']
    database = DATABASES['default']['NAME']
    username = DATABASES['default']['USER']
    password = DATABASES['default']['PASSWORD']

    # connection parameters
    params = 'DRIVER=' + driver + ';' \
        'SERVER=' + server + ';' \
        'DATABASE=' + database + ';' \
        'UID=' + username + ';' \
        'PWD=' + password + ';'

    conn_params = urllib.parse.quote_plus(params)
    dbconn = create_engine('mssql+pyodbc:///?odbc_connect=%s' %
                           conn_params, fast_executemany=False)

    autocommit_engine = dbconn.execution_options(isolation_level="AUTOCOMMIT")
    return dbconn
engine = connect_database()

In [41]:
""" retrieve all site name from database """

query = f"SELECT SiteName as Site FROM SiteMaster"
result = engine.execute(query).fetchall()
site_list = [row['Site'] for row in result]

print("site_list", site_list)

site_list ['Fegersheim', 'Sesto', 'IDAP_IPM', 'NALO', 'Indy_API', 'Branchburg', 'Alcobendas', 'Kinsale', 'Seishin', 'Suzhou', 'PR1', 'PR05', 'Limerick_API', 'Concord', 'RTP', 'DC_North_Carolina', 'Lebanon_API', 'Lebanon_AT', 'IDAP', 'IPM', 'TST_Central', 'TST_API', 'TST_Expansion', 'TST_Other', 'Affiliate', 'NewFunctionTestSite']


In [44]:
""" make the site name is correct and match the one in from database """

unmatched_set = set()

for value in df_site['Site']:
    if value not in site_list:
        unmatched_set.add(value)

print("unmatched_set", unmatched_set)        

unmatched_set set()


In [43]:
""" replace the unmatch value with the correct site name, and repeat the above step to make sure there is no unmatched site name """

df_site['Site'] = df_site['Site'].replace({'Feg': 'Fegersheim'})

In [45]:
""" for all df, add site column, and repeat the whole df for each site """

df_list = []

for site in site_list:
    df_copy = df_all.copy()
    df_copy['Site'] = site
    df_list.append(df_copy)

df_all = pd.concat(df_list, ignore_index=True)
print('df_all', df_all)

df_all       System MvtType                       Description MvtInScopeTptCalc  \
0        EWM    1010                           Putaway                 Y   
1        EWM    1011      Putaway with Storage Process                 N   
2        EWM    1012            Putaway (Distributive)                 Y   
3        EWM    1013  Putaway with VAS Storage Process                 N   
4        EWM    1013                   Putaway to B108                 N   
...      ...     ...                               ...               ...   
29245     WM     ZY2              RE all acct.assignB5                 N   
29246     WM     ZZ1                  GR goods receipt                 N   
29247     WM     ZZ2              GR for PO   reversal                 N   
29248     WM     ZZ3                  GR goods receipt                 N   
29249     WM     ZZ4                 GR TP dely w. tax                 N   

                      Site  
0               Fegersheim  
1               Fegers

In [47]:
""" insert into database """

datatable = 'LKU_MvtType_InScope'

# df_site.to_sql(datatable, engine, if_exists='append', index=False)
df_all.to_sql(datatable, engine, if_exists='append', index=False)

print("Done")

Done
